In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.metrics import f1_score
import time
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from create_spectrogram_and_time_series_dataset_use import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstarping_partitions_full_int_q,
    )

2024-06-20 11:21:25.417957: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 11:21:25.457024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 11:21:25.457050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 11:21:25.457901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 11:21:25.463167: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 11:21:25.463931: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-06-20 11:21:28.525247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 11:21:28.525630: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-20 11:21:28.616978: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-20 11:21:28.620449: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [8]:
BASE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/squeezenet/squeezenet_spec_16kHz_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 184, 80, 1)]         0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 92, 40, 96)           4800      ['input_1[0][0]']             
                                                                                                  
 maxpool1 (MaxPooling2D)     (None, 45, 19, 96)           0         ['conv1[0][0]']               
                                                                                                  
 fire2_squeeze (Conv2D)      (None, 45, 19, 16)           1552      ['maxpool1[0][0]']            
                                                                                              

In [9]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(baseline_model, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras")

Validation dataset:
44/44 [==============================] - 4s 78ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.77%
Recall: 99.36%
Precision: 97.10%
F1-score: 98.22%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9821834731737121

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9809484989817016

Test dataset:
44/44 [==============================] - 4s 86ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9778618945971255

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9777210510822794


In [10]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np

In [11]:
for i in x_val_np:
    print(i.shape)
    break
    

(184, 80, 1)


In [12]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

tflite_model_quant = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpsn0_u3f8/assets


INFO:tensorflow:Assets written to: /tmp/tmpsn0_u3f8/assets
/home/polina/newname/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-20 11:24:10.095875: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-20 11:24:10.095959: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-20 11:24:10.096568: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpsn0_u3f8
2024-06-20 11:24:10.102673: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-20 11:24:10.102730: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpsn0_u3f8
2024-06-20 11:24:10.117292: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass

In [13]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [14]:
import pathlib

tflite_models_dir = pathlib.Path("spectrogram_models_from_notebooks/squeezenet")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"squeezenet_spec_16kHz_full_int_q.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

826920

In [15]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_quant_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_quant_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_quant_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_quant_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_quant_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_quant_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_quant_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_quant_file), "KB")

Validation dataset:


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy: 98.41%
Recall: 97.25%
Precision: 98.08%
F1-score: 97.66%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9760380311067124

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.976840710931716

Test dataset:
Accuracy: 98.21%
Recall: 97.82%
Precision: 96.76%
F1-score: 97.29%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9722449161503498

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9773114137330187

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.01 seconds
Max: 0.022 seconds
Min: 0.009 seconds
File size: 807.539 Kilobytes
